# Metric analysis

The folowing notebook presents **statistical** and **machine learning based** methods to analyze and select the most meaningfull features that predict the human score within a dataset of metrics

### TODO


https://chatgpt.com/c/67963c90-aca8-8001-8479-7efaac72c1ba